# Package installation 

Execute only if you are usng Goolge Colab. If you are using your own local environment make sure you installed the dependecies first

In [ ]:
! pip install snntorch
! pip install neurobench
! pip install tonic
! pip install nir==1.0.4
! pip install numpy --upgrade

# Import packages

In [1]:
import torch
import torch.nn as nn
import snntorch as snn
import numpy as np
from snntorch.import_nir import import_from_nir
import nir

# Datased definition

In [ ]:
from tonic import datasets, transforms

def get_data_loaders(batch_size):
    
    transform = transforms.Compose([
        transforms.ToFrame(sensor_size=(34, 34, 2), n_time_bins=30, include_incomplete=True),
        lambda x: torch.from_numpy(x.astype(np.float32)),
    ])

    trainset = datasets.NMNIST('./data', train=True, transform=transform)
    testset = datasets.NMNIST('./data', train=False, transform=transform)

    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=0,
        drop_last=True, pin_memory=True
    )
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, num_workers=0,
        drop_last=True, pin_memory=True
    )

    return trainloader, testloader

/Users/benedettoleto/PycharmProjects/inNuCE_BRANDY-2025/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# NIR

Import model using NIR and convert the model into a torch based Neural Network using snnTorch

In [3]:
graph = nir.read("cnn_sinabs.nir")
graph.nodes.keys()
net = import_from_nir(graph)
print(net)

replace rnn subgraph with nirgraph
GraphExecutor(
  (1): Conv2d(2, 8, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (10): Leaky()
  (11): Flatten(start_dim=0, end_dim=-1)
  (2): Leaky()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (4): Leaky()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (6): Leaky()
  (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(np.int64(2), np.int64(2)), padding=(np.int64(1), np.int64(1)))
  (8): Leaky()
  (9): Conv2d(64, 10, kernel_size=(3, 3), stride=(np.int64(1), np.int64(1)))
  (input): Identity()
  (output): Identity()
)


Once the model has benn converted in snnTorch inizialize the neurons membrane potential 

In [4]:
modules = [e.elem for e in net.get_execution_order()]

# init all I&F neurons
mem_dict = {}
for idx, module in enumerate(modules):
  if isinstance(module, snn.Leaky):
    module.mem = module.init_leaky()

# NeuroBench

Benchmarking using this tool to extract useful statistics from our model

In [7]:
from neurobench.processors.postprocessors import ChooseMaxCount
from neurobench.benchmarks import Benchmark
from neurobench.models import SNNTorchModel

from neurobench.metrics.workload import (
    ActivationSparsity,
    MembraneUpdates,
    SynapticOperations,
    ActivationSparsityByLayer,
)
from neurobench.metrics.static import (
    ParameterCount,
    Footprint,
    ConnectionSparsity,
)

_, testloader = get_data_loaders(batch_size=32)


model = SNNTorchModel(net, custom_forward=False)

static_metrics = [ParameterCount, Footprint, ConnectionSparsity]
workload_metrics = [ActivationSparsity, ActivationSparsityByLayer,MembraneUpdates, SynapticOperations]

benchmark = Benchmark(
    model, testloader, [], [], [static_metrics, workload_metrics]
)
results = benchmark.run(verbose=False)

print(results)

Running benchmark


  0%|          | 1/312 [00:00<01:00,  5.17it/s]

Error running workload metric 'SynapticOperations': Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment.  If you were attempting to deepcopy a module, this may be because of a torch.nn.utils.weight_norm usage, see https://github.com/pytorch/pytorch/pull/103001


100%|██████████| 312/312 [01:43<00:00,  3.00it/s]

{'ParameterCount': 30226, 'Footprint': 760236, 'ConnectionSparsity': 0.0, 'ActivationSparsity': 0.931641162728608, 'ActivationSparsityByLayer': {'10': 0.992018563034188, '2': 0.912668631979564, '4': 0.9143628313532104, '6': 0.9769774055154915, '8': 0.9956004607371794}, 'MembraneUpdates': 89915.54477163461, 'SynapticOperations': {'Effective_MACs': 49366.6422829582, 'Effective_ACs': 325954.966840836, 'Dense': 6983059.458400321}}
